### Setup libraries
Let's start with some imports. 


In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division

# autoreload modules before executing
%load_ext autoreload
%autoreload 2

import glob
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import importlib
import os
import shutil
import pandas as pd
import pprint
import sys
# Setup pretty printing
pp = pprint.PrettyPrinter(indent=4)


font = {'family' : 'DejaVu Sans',
        'size'   : 20}

matplotlib.rc('font', **font)
%matplotlib inline



from tifffile import imread
from csbdeep.utils import download_and_extract_zip_file, plot_some
from csbdeep.data import RawData, create_patches

# importing some utilities from microDL in the first half.
# Add microdl path to sys
module_path = os.path.abspath(os.path.join('..'))
print("System path: "+module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
    
import micro_dl.utils.image_utils as im_utils
import micro_dl.utils.masks as mask_utils
import micro_dl.utils.normalize as norm_utils
TOP_DIR = os.path.expanduser('~/CompMicro/projects/virtualstaining/MBL_DL_image_translation')
INPUT_DIR = os.path.join(TOP_DIR, 'data/')
MODEL_DIR = os.path.join(TOP_DIR, 'care_model')
patch_file = INPUT_DIR+'phase_dna_care.npz'



## A. Prepare dataset for training

### Paths

In [ ]:

positions = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# Copy the data in a format expected by CARE pipeline. 
# CARE data generator requires the source and target files to be saved in different formats with matching filenames.

# subset retardance images
if not os.path.exists(INPUT_DIR + 'care_retardance'):
    os.mkdir(INPUT_DIR + 'care_retardance')
for pos in positions:
    srcfiles= glob.glob(INPUT_DIR + 'img_Retardance_*p{:03d}'.format(pos)+ '*')
    dstfiles= [path.replace('img_Retardance_','care_retardance/img_') for path in srcfiles]
    for src, dst in zip(srcfiles,dstfiles):
        shutil.copy(src,dst)

# subset phase images
if not os.path.exists(INPUT_DIR + 'care_phase'):
    os.mkdir(INPUT_DIR + 'care_phase')
for pos in positions:
    srcfiles= glob.glob(INPUT_DIR + 'img_phase_*p{:03d}'.format(pos)+ '*')
    dstfiles= [path.replace('img_phase_','care_phase/img_') for path in srcfiles]
    for src, dst in zip(srcfiles,dstfiles):
        shutil.copy(src,dst)
        
# subset F-actin images
if not os.path.exists(INPUT_DIR + 'care_factin'):
    os.mkdir(INPUT_DIR + 'care_factin')
        
for pos in positions:
    srcfiles= glob.glob(INPUT_DIR + 'img_568_*p{:03d}'.format(pos)+ '*')
    dstfiles= [path.replace('img_568_','care_factin/img_') for path in srcfiles]
    for src, dst in zip(srcfiles,dstfiles):
        shutil.copy(src,dst)

# subset DNA images
if not os.path.exists(INPUT_DIR + 'care_dna'):
    os.mkdir(INPUT_DIR + 'care_dna')
        
for pos in positions:
    srcfiles= glob.glob(INPUT_DIR + 'img_405_*p{:03d}'.format(pos)+ '*')
    dstfiles= [path.replace('img_405_','care_dna/img_') for path in srcfiles]
    for src, dst in zip(srcfiles,dstfiles):
        shutil.copy(src,dst)

### Explore input and target images <a class="anchor" id="explore"></a>

The goal of this exercise is to translate 2D slices of phase or retardance image into 2D slices of fluorescene images of F-actin and DNA. Let's start by looking at some example images.

In [ ]:
# import the libaries again if the plots don't display properly
img_name = 'img_t000_p003_z010.tif'
actin_path = os.path.join(INPUT_DIR, 'care_factin/', img_name) #F-actin was imaged with Alexa Fluor 568 using 468nm excitation wavelength.
dna_path=os.path.join(INPUT_DIR, 'care_dna/', img_name) # DNA was imaged with Hoechst using 405nm excitation wavelength.
input_path=os.path.join(INPUT_DIR,'care_retardance/', img_name) # the phase and polarization were imaged using 530nm wavelength.

actin = imread(actin_path)
# This will clip the top and bottom 1% of intensitites
actin = norm_utils.hist_clipping(actin, 1, 99)
dna = imread(dna_path)
dna = norm_utils.hist_clipping(dna, 1, 99)
im_input = imread(input_path) 
im_input = norm_utils.hist_clipping(im_input, 0.8, 99.5) 
fig, ax = plt.subplots(1, 3)
fig.set_size_inches(20, 15)
ax = ax.flatten()
ax[0].imshow(im_input, cmap='gray')
ax[0].set_title('Input phase',fontsize=20)
ax[1].imshow(actin, cmap='gray')
ax[1].set_title('Target F-actin',fontsize=20)
ax[2].imshow(dna, cmap='gray')
ax[2].set_title('Target DNA',fontsize=20)
for a in ax: a.axis('off')

plt.show()
plt.close('all')

### Create a data generator and patches

In [ ]:
from csbdeep.data import no_background_patches, norm_percentiles, sample_percentiles

raw_data = RawData.from_folder (
    basepath    = INPUT_DIR,
    source_dirs = ['care_phase'],
    target_dir  = 'care_dna',
    axes        = 'YX',
)

X, Y, XY_axes = create_patches (
    raw_data            = raw_data,
    patch_size          = (256,256),
    patch_filter        = no_background_patches(0.5),
    n_patches_per_image = 128,
    save_file           = patch_file,
)
# TODO: include augmentations (transforms = ???)

In [ ]:
assert X.shape == Y.shape
print("shape of X,Y =", X.shape)
print("axes  of X,Y =", XY_axes)


for i in range(2):
    plt.figure(figsize=(16,4))
    sl = slice(8*i, 8*(i+1)), 0
    plot_some(X[sl],Y[sl],title_list=[np.arange(sl[0].start,sl[0].stop)])
    plt.show()
None;
## TODO: plot in grayscale.

### Checkpoint A

## B. 2D virtual staining (slice$\rightarrow$slice) using CARE <a class="anchor" id="pilot"></a>

### Load training data and configure

In [ ]:
from csbdeep.utils import axes_dict, plot_some, plot_history
from csbdeep.utils.tf import limit_gpu_memory
from csbdeep.io import load_training_data
from csbdeep.models import Config, CARE


In [ ]:
(X,Y), (X_val,Y_val), axes = load_training_data(patch_file, validation_split=0.1, verbose=False)

c = axes_dict(axes)['C']
n_channel_in, n_channel_out = X.shape[c], Y.shape[c]

In [ ]:
config = Config(axes, n_channel_in, n_channel_out, unet_kern_size=3, train_batch_size=16, train_steps_per_epoch=10, unet_n_depth = 3, train_epochs = 30, unet_residual = False)
print(config)
vars(config)

In [ ]:
model = CARE(config, 'phase2DNA_bath16_depth3', basedir=MODEL_DIR) 

In [ ]:
model.keras_model.summary()
model

### Time to train. 

Try restarting the notebook kernel if the gpu memory is occupied and you run into errors about "can't creat training session".
Training 6 epochs should take no more than 5 minutes if you're on a GPU.

In [ ]:
history = model.train(X,Y, validation_data=(X_val,Y_val))

We've completed our first training. Let's take a look at what happened during training by opening a history log file.

In [ ]:
print(sorted(list(history.history.keys())))
plt.figure(figsize=(16,5))
plot_history(history,['loss','val_loss'],['mse','val_mse','mae','val_mae']);

In [ ]:
plt.figure(figsize=(20,12))
_P = model.keras_model.predict(X_val[:5])
if config.probabilistic:
    _P = _P[...,:(_P.shape[-1]//2)]
plot_some(X_val[:5],Y_val[:5],_P,pmax=99.5)
plt.suptitle('5 example validation patches\n'      
             'top row: input (source),  '          
             'middle row: target (ground truth),  '
             'bottom row: predicted from source');

In [ ]:
x_val_min = np.min(X_val[:])
x_val_max = np.max(X_val[:])
print(f'size of validation input: {X_val[:5].shape}, range of validation input: [{x_val_min}, {x_val_max}]')

### Checkpoint B

## C. Prediction on test position not seen by model

In [ ]:
# Read test image and scale the uint16 image to match the dynamic range of validation set.

im_path = os.path.join(INPUT_DIR, 'img_phase_t000_p011_z013.tif')
test_input = imread(im_path).astype('float32')
# test_input -= np.mean(test_input[:])
# test_input /= np.max(test_input[:])
# test_input *= x_val_max
# prediction = model.keras_model.predict(test_input.reshape(1,2048,2048,1)) # Note that the input needs to be reshaped in (batch, Y, X, C) order.
# prediction = prediction.reshape(2048,2048)

prediction = model.predict(test_input, axes = 'YX')

im_path = os.path.join(INPUT_DIR, 'img_405_t000_p011_z013.tif')
target = imread(im_path)

print(f'input range: {np.min(test_input[:])}')

fig, ax = plt.subplots(1, 3)
fig.set_size_inches(20, 15)
ax = ax.flatten()
for a in ax: a.axis('off')
    
ax[0].imshow(norm_utils.hist_clipping(test_input, 1, 99), 
             cmap='gray');
ax[0].set_title('Input', fontsize=20)

ax[1].imshow(norm_utils.hist_clipping(target, 1, 99), 
             cmap='gray'); 
ax[1].set_title('Target', fontsize=20)

ax[2].imshow(norm_utils.hist_clipping(prediction, 1, 99), 
             cmap='gray'); 
ax[2].set_title('Prediction', fontsize=20)

plt.show()
plt.close('all')

### Checkpoint C